
# Consultas Vectoriales en LanceDB

## Introducción
Las consultas vectoriales permiten encontrar elementos similares basados en representaciones vectoriales de datos, como texto, imágenes o audio.  **LanceDB** es una base de datos optimizada para búsquedas vectoriales, ideal para casos de uso como recuperación de información, sistemas de recomendación y análisis multimodal.

En este notebook aprenderemos a:
1. Generar embeddings para distintos tipos de datos: texto, imágenes, audio y multimodal.
2. Almacenar y consultar estos embeddings en LanceDB.
3. Realizar búsquedas vectoriales y combinadas.

---



## 1. Configuración Inicial

Primero, configuraremos el entorno para trabajar con LanceDB y las librerías necesarias para generar embeddings de texto, imágenes y audio.


In [16]:
# Importar bibliotecas necesarias
import lancedb
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from PIL import Image  # Para manejo de imágenes
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
import librosa  
import os  
import pyarrow as pa
# Confirmar que las bibliotecas se importaron correctamente 
print("Bibliotecas importadas correctamente.")






Bibliotecas importadas correctamente.


In [12]:
# Configurar LanceDB
db_path = "data/vectorial_db"  # Ruta donde se almacenará la base de datos

# Crear el directorio si no existe
os.makedirs(db_path, exist_ok=True)

# Conectar a LanceDB
db = lancedb.connect(db_path)

print(f"Base de datos configurada en: {db_path}")


Base de datos configurada en: data/vectorial_db



## 2. Generar Embeddings de Texto

Utilizaremos **sentence-transformers** para convertir texto en representaciones vectoriales que puedan almacenarse y consultarse en LanceDB.


In [14]:
# Cargar modelo para embeddings de texto
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Datos de ejemplo
text_data = [
    "El cielo es azul y hermoso.",
    "Los gatos son mascotas populares.",
    "La inteligencia artificial está transformando el mundo.",
    "Las flores son coloridas y fragantes.",
    "Viajar es una experiencia enriquecedora."
]

# Generar embeddings
text_embeddings = text_model.encode(text_data)
text_embeddings = [embedding.astype(np.float32).tolist() for embedding in text_embeddings]  # Convertir a float32

# Crear DataFrame con los datos y embeddings
df_text = pd.DataFrame({
    "texto": text_data,
    "embedding": text_embeddings
})

# Convertir a una tabla Arrow con el esquema simulado usando pa.list_
schema = pa.schema([
    ("texto", pa.string()),  # Columna de texto
    ("embedding", pa.list_(pa.float32(), list_size=len(text_embeddings[0])))  # Lista con tamaño esperado
])

# Crear la tabla Arrow
table = pa.Table.from_pandas(df_text, schema=schema)

# Guardar la tabla en LanceDB
tabla_text = db.create_table("text_embeddings", data=table, mode="overwrite")
print("Tabla 'text_embeddings' creada con éxito con pa.list_.")




Tabla 'text_embeddings' creada con éxito con pa.list_.



## 3. Generar Embeddings de Imágenes

Usaremos un modelo preentrenado como CLIP para convertir imágenes en vectores representativos.


In [17]:
# Configuración del modelo y transformaciones
image_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Usar el parámetro 'weights' para evitar advertencias
image_model.eval()

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Ruta a la carpeta de imágenes
image_dir = "data/images"

# Procesar cada imagen
image_embeddings = []
image_data = []

for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        img_path = os.path.join(image_dir, image_file)
        img = Image.open(img_path).convert("RGB")
        img_tensor = image_transform(img).unsqueeze(0)

        # Generar embedding
        with torch.no_grad():
            embedding = image_model(img_tensor).squeeze().numpy().astype('float32')

        image_data.append(image_file)  # Nombre de la imagen
        image_embeddings.append(embedding.tolist())  # Embedding como lista

# Crear DataFrame con los datos y embeddings
df_images = pd.DataFrame({
    "imagen": image_data,  # Nombre de la imagen
    "embedding": image_embeddings  # Embedding de la imagen
})

print("Embeddings generados para imágenes:")
print(df_images.head())

# Guardar en LanceDB
# Crear esquema usando pa.list_
schema = pa.schema([
    ("imagen", pa.string()),  # Nombre de la imagen
    ("embedding", pa.list_(pa.float32(), list_size=len(image_embeddings[0])))  # Lista de tamaño esperado
])

# Crear tabla Arrow
table = pa.Table.from_pandas(df_images, schema=schema)

# Guardar la tabla en LanceDB
tabla_images = db.create_table("image_embeddings", data=table, mode="overwrite")
print("Tabla 'image_embeddings' creada con éxito.")




Embeddings generados para imágenes:
         imagen                                          embedding
0  imagen_1.jpg  [4.202572822570801, -2.4697437286376953, 1.471...
1  imagen_6.jpg  [2.447737216949463, 1.3279236555099487, -0.803...
2  imagen_5.jpg  [3.017625570297241, 1.0321232080459595, 0.4439...
3  imagen_4.jpg  [5.526756286621094, 7.365933895111084, 1.59381...
4  imagen_3.jpg  [2.2661871910095215, -1.88070547580719, 0.1356...
Tabla 'image_embeddings' creada con éxito.


[2024-12-01T02:54:38Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/vectorial_db/image_embeddings.lance, it will be created


## 4. Generar Embeddings de Audio

Usaremos **librosa** para extraer características de audio y convertirlas en vectores.


In [18]:
# Ruta a la carpeta de audios
audio_dir = "data/audios"  

# Procesar cada archivo de audio
audio_data = []
audio_embeddings = []

# Configuración de MFCC
n_mfcc = 13  # Tamaño fijo para los embeddings
embedding_length = n_mfcc  # Aseguramos un tamaño fijo

for audio_file in os.listdir(audio_dir):
    if audio_file.endswith(".wav") or audio_file.endswith(".mp3"):
        audio_path = os.path.join(audio_dir, audio_file)
        
        # Cargar el audio
        y, sr = librosa.load(audio_path, sr=None)  # sr=None conserva la tasa original

        # Extraer MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        # Generar embedding de tamaño fijo (promediando)
        mfcc_mean = np.mean(mfcc, axis=1).astype(np.float32)  # Tamaño fijo de n_mfcc

        # Guardar los datos
        audio_data.append(audio_file)  # Nombre del archivo
        audio_embeddings.append(mfcc_mean.tolist())  # Embedding como lista

# Crear DataFrame con los datos y embeddings
df_audio = pd.DataFrame({
    "audio": audio_data,
    "embedding": audio_embeddings
})

print("Embeddings generados para audios:")
print(df_audio.head())

# Crear esquema usando pa.list_ para guardar en LanceDB
schema = pa.schema([
    ("audio", pa.string()),  # Nombre del archivo de audio
    ("embedding", pa.list_(pa.float32(), list_size=embedding_length))  # Embedding de tamaño fijo
])

# Convertir DataFrame a tabla Arrow
table = pa.Table.from_pandas(df_audio, schema=schema)

# Guardar en LanceDB
tabla_audio = db.create_table("audio_embeddings", data=table, mode="overwrite")
print("Tabla 'audio_embeddings' creada con éxito.")





Embeddings generados para audios:
          audio                                          embedding
0  esponja4.mp3  [-185.07440185546875, 130.5316619873047, -56.4...
1  esponja2.mp3  [-175.2745819091797, 88.03491973876953, -27.00...
2  esponja1.mp3  [-103.59510040283203, 124.67962646484375, -31....
3  esponja3.mp3  [-105.74647521972656, 64.4498062133789, -136.2...
Tabla 'audio_embeddings' creada con éxito.


[2024-12-01T02:55:05Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/vectorial_db/audio_embeddings.lance, it will be created


# Consultas Vectoriales

Realizaremos búsquedas vectoriales en LanceDB para encontrar elementos similares.


## Consultas sobre los embeddings de texto

In [ ]:
# Consulta 1: Tema general
query_text_1 = "Las flores tienen colores brillantes."  # Texto de consulta
query_embedding_1 = text_model.encode([query_text_1])[0].astype(np.float32)  # Generar embedding

# Buscar en la tabla
results_text_1 = tabla_text.search(query_embedding_1, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar resultados
print("Resultados de búsqueda para texto (consulta 1):")
print(results_text_1)



Resultados de búsqueda para texto (consulta 1):
                                      texto  \
0     Las flores son coloridas y fragantes.   
1  Viajar es una experiencia enriquecedora.   
2               El cielo es azul y hermoso.   

                                           embedding  _distance  
0  [-0.016914625, -0.012992999, -0.02974755, 0.02...   0.451207  
1  [0.06420154, 0.026209284, -0.03604845, 0.05888...   1.035577  
2  [-0.003502195, 0.08340518, -0.04015484, -0.000...   1.148967  


In [6]:
# Consulta 2: Tema específico
query_text_2 = "La inteligencia artificial cambiará el futuro."  # Texto de consulta
query_embedding_2 = text_model.encode([query_text_2])[0].astype(np.float32)  # Generar embedding

# Buscar en la tabla
results_text_2 = tabla_text.search(query_embedding_2, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar resultados
print("Resultados de búsqueda para texto (consulta 2):")
print(results_text_2)

Resultados de búsqueda para texto (consulta 2):
                                               texto  \
0  La inteligencia artificial está transformando ...   
1                        El cielo es azul y hermoso.   
2           Viajar es una experiencia enriquecedora.   

                                           embedding  _distance  
0  [-0.049482808, 0.057707522, -0.0005006774, -0....   0.726331  
1  [-0.003502195, 0.08340518, -0.04015484, -0.000...   1.063330  
2  [0.06420154, 0.026209284, -0.03604845, 0.05888...   1.256500  


## Consultas sobre los embeddings de imágenes

In [15]:
from PIL import Image
import torch

# Crear una imagen de consulta (colores suaves)
query_image_1 = Image.new("RGB", (224, 224), (80, 120, 160))  # Color RGB suave
query_image_tensor_1 = image_transform(query_image_1).unsqueeze(0)

# Generar embedding para la imagen de consulta
with torch.no_grad():
    query_image_embedding_1 = image_model(query_image_tensor_1).squeeze().numpy().astype('float32')

# Realizar la búsqueda en LanceDB
results_image_1 = tabla_images.search(query_image_embedding_1, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar los resultados
print("Resultados de búsqueda para la consulta 1:")
print(results_image_1)



Resultados de búsqueda para la consulta 1:
         imagen                                          embedding  \
0  imagen_5.jpg  [3.0176253, 1.0321219, 0.4439306, -0.8215103, ...   
1  imagen_3.jpg  [2.2661877, -1.8807054, 0.13566248, -1.3516074...   
2  imagen_1.jpg  [4.202572, -2.469743, 1.4718562, 0.99616164, 2...   

     _distance  
0  3776.850830  
1  4464.903809  
2  6996.564941  


In [19]:
from PIL import Image

# Cargar una imagen de ejemplo desde el disco
query_image_path_2 = "data/images/imagen_6.jpg" 
query_image_2 = Image.open(query_image_path_2).convert("RGB")
query_image_tensor_2 = image_transform(query_image_2).unsqueeze(0)

# Generar embedding para la imagen de consulta
with torch.no_grad():
    query_image_embedding_2 = image_model(query_image_tensor_2).squeeze().numpy().astype('float32')

# Realizar la búsqueda en LanceDB
results_image_2 = tabla_images.search(query_image_embedding_2, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar los resultados
print("Resultados de búsqueda para la consulta 2:")
print(results_image_2)


Resultados de búsqueda para la consulta 2:
         imagen                                          embedding  \
0  imagen_6.jpg  [2.4477372, 1.3279237, -0.8030599, -3.3387642,...   
1  imagen_5.jpg  [3.0176256, 1.0321232, 0.44393137, -0.8215101,...   
2  imagen_1.jpg  [4.202573, -2.4697437, 1.4718571, 0.9961633, 2...   

     _distance  
0     0.000000  
1  5792.356934  
2  6944.045410  


In [20]:

# Seleccionar un audio de la tabla para la consulta
query_audio_path = "data/audios/esponja4.mp3"  
y, sr = librosa.load(query_audio_path, sr=None)

# Extraer características (MFCC) y generar embedding
query_mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
query_audio_embedding = np.mean(query_mfcc, axis=1).astype(np.float32)

# Realizar la búsqueda en LanceDB
results_audio = tabla_audio.search(query_audio_embedding, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar resultados
print("Resultados de búsqueda para la consulta 1:")
print(results_audio)





Resultados de búsqueda para la consulta 1:
          audio                                          embedding  \
0  esponja4.mp3  [-185.0744, 130.53166, -56.47632, 24.994247, -...   
1  esponja2.mp3  [-175.27458, 88.03492, -27.005796, 20.219791, ...   
2  esponja1.mp3  [-103.5951, 124.67963, -31.355947, 32.714035, ...   

     _distance  
0     0.000000  
1  4658.028809  
2  8948.547852  


In [ ]:
# Seleccionar un audio de la tabla para la consulta
query_audio_path = "data/audios/esponja1.mp3"  
y, sr = librosa.load(query_audio_path, sr=None)

# Extraer características (MFCC) y generar embedding
query_mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
query_audio_embedding = np.mean(query_mfcc, axis=1).astype(np.float32)

# Realizar la búsqueda en LanceDB
results_audio = tabla_audio.search(query_audio_embedding, vector_column_name="embedding").limit(3).to_pandas()

# Mostrar resultados
print("Resultados de búsqueda para la consulta 1:")
print(results_audio)


Resultados de búsqueda para la consulta 1:
          audio                                          embedding  \
0  esponja1.mp3  [-103.5951, 124.67963, -31.355947, 32.714035, ...   
1  esponja2.mp3  [-175.27458, 88.03492, -27.005796, 20.219791, ...   
2  esponja4.mp3  [-185.0744, 130.53166, -56.47632, 24.994247, -...   

     _distance  
0     0.000000  
1  7900.018555  
2  8948.547852  
